In [1]:
import pandas as pd

# Excel dosyalarını yükleyin
file_path1 = r'C:\Users\ilayd\OneDrive\Masaüstü\ffilm.xlsx'
file_path2 = r'C:\Users\ilayd\OneDrive\Masaüstü\filmm.xlsx'
file_path3 = r'C:\Users\ilayd\OneDrive\Masaüstü\movie.xlsx'

# Dosyaların sayfalarını okuyun
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)
df3 = pd.read_excel(file_path3)

# Üçüncü dosya (movie) ve ikinci dosyayı (filmm) movieId'ye göre birleştirin
merged_df1 = pd.merge(df3, df2, on='movieId', how='left')

# Şimdi bu birleştirilmiş DataFrame ile birinci dosyayı (ffilm) userId ve movieId'ye göre birleştirin
final_merged_df = pd.merge(merged_df1, df1, on=['userId', 'movieId'], how='left')

# Eksik userId'leri belirleyin
# Eksik userId'leri belirlemek için, df3'te bulunan userId'leri final_merged_df'de ararız
missing_user_ids = df3[~df3['userId'].isin(final_merged_df['userId'])]

# Eksik userId'lerin eksik olduğu satırları belirleyin
if not missing_user_ids.empty:
    print("Eksik olan userId'ler:")
    print(missing_user_ids)
    
    # Eksik olan userId'leri ayrı bir Excel dosyasına kaydedin
    missing_user_ids.to_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\missing_user_ids.xlsx', index=False)

# Verileri userId'ye göre sıralayın
sorted_df = final_merged_df.sort_values(by='userId')

# Sonuçları yeni bir Excel dosyasına kaydedin
sorted_df.to_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\final_merged_output.xlsx', index=False)

print("Veriler sıralandı ve dosyaya kaydedildi.")

Veriler sıralandı ve dosyaya kaydedildi.


In [2]:
# İlk 5 satırı bul
sorted_df.head()

,userId,movieId,rating,timestamp_x,title,genres,tag,timestamp_y
0,1,1,40,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
147,1,2329,50,964983263,American History X (1998),Crime|Drama,NaN,NaN
148,1,2338,20,964983546,I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller,NaN,NaN
149,1,2353,50,964983861,Enemy of the State (1998),Action|Thriller,NaN,NaN
150,1,2366,40,964982462,King Kong (1933),Action|Adventure|Fantasy|Horror,NaN,NaN


In [3]:
# Son 5 satırı bul
sorted_df.tail()

,userId,movieId,rating,timestamp_x,title,genres,tag,timestamp_y
101804,610,6387,35,1479542038,Once a Thief (Zong heng si hai) (1991),Action|Comedy|Crime|Thriller,NaN,NaN
101803,610,6383,25,1493846084,"2 Fast 2 Furious (Fast and the Furious 2, The)...",Action|Crime|Thriller,NaN,NaN
101802,610,6378,35,1493844983,"Italian Job, The (2003)",Action|Crime,NaN,NaN
101820,610,6708,35,1493847441,Matchstick Men (2003),Comedy|Crime|Drama,NaN,NaN
102676,610,170875,30,1493846415,The Fate of the Furious (2017),Action|Crime|Drama|Thriller,NaN,NaN


In [4]:
# Filmlerin yorum sayılarının belirlenmesi
# (sorted_df veri setinde "title" sütunundaki her bir başlığın kaç kez tekrarlandığını hesaplar
# Sütunları yeniden adlandırır; 'index' aslında film başlığıdır, 'title' ise bu başlığın kaç kez tekrarlandığını gösterir)

comment_counts = pd.DataFrame(sorted_df["title"].value_counts()).reset_index()
comment_counts.columns = ['title', 'comment_count']

# Yorum sayısına göre 10 film
print(comment_counts.head(10)) 

                                       title  comment_count
0                        Pulp Fiction (1994)            484
1                        Forrest Gump (1994)            335
2           Shawshank Redemption, The (1994)            319
3           Silence of the Lambs, The (1991)            283
4                         Matrix, The (1999)            280
5                          Fight Club (1999)            268
6  Star Wars: Episode IV - A New Hope (1977)            262
7                          Braveheart (1995)            245
8                       Jurassic Park (1993)            238
9          Terminator 2: Judgment Day (1991)            229


In [5]:
# Az yorum alan filmlerin belirlenmesi (örneğin, 100'den az yorum alan filmler)
rare_movies = comment_counts[comment_counts["comment_count"] <= 100]["title"]

# Az yorum alan filmlerin çıkartılması
common_movies = sorted_df[~sorted_df["title"].isin(rare_movies)]

# Sonuçları kontrol edin
print(f"Yorum sayısı 100'den fazla olan film sayısı: {common_movies['title'].nunique()}")
print(common_movies.head())

Yorum sayısı 100'den fazla olan film sayısı: 137
     userId  movieId  rating  timestamp_x  \
0         1        1      40    964982703   
147       1     2329      50    964983263   
166       1     2571      50    964981888   
171       1     2628      40    964980523   
121       1     2028      40    964981888   

                                                title  \
0                                    Toy Story (1995)   
147                         American History X (1998)   
166                                Matrix, The (1999)   
171  Star Wars: Episode I - The Phantom Menace (1999)   
121                        Saving Private Ryan (1998)   

                                          genres  tag  timestamp_y  
0    Adventure|Animation|Children|Comedy|Fantasy  NaN          NaN  
147                                  Crime|Drama  NaN          NaN  
166                       Action|Sci-Fi|Thriller  NaN          NaN  
171                      Action|Adventure|Sci-Fi  NaN        

In [6]:
#Kullanıcı Film matrisinin oluşturulma
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

user_movie_df.head()

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),Apocalypse Now (1979),...,True Lies (1994),"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),Up (2009),"Usual Suspects, The (1995)",WALLÂ·E (2008),Waterworld (1995),Willy Wonka & the Chocolate Factory (1971),X-Men (2000)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,40.0,NaN,NaN,50.0,50.0,NaN,40.0,...,NaN,NaN,NaN,30.0,NaN,50.0,NaN,NaN,50.0,50.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,40.0,NaN,NaN,NaN,50.0,NaN,NaN,NaN,...,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,40.0,NaN
5,NaN,30.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN


In [7]:
#Birincil filmin belirlenmesi
movie_name = "X-Men (2000)"

#Filmin seçilmesi
movie_name = user_movie_df[movie_name]

#Film ile korelasyonu en yüksek olan filmlerin sıralanması
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
X-Men (2000)                                                        1.000000
Star Trek: Generations (1994)                                       0.571793
Batman Begins (2005)                                                0.475671
Harry Potter and the Chamber of Secrets (2002)                      0.473868
Men in Black (a.k.a. MIB) (1997)                                    0.472008
Spider-Man (2002)                                                   0.464925
Stargate (1994)                                                     0.455950
Die Hard: With a Vengeance (1995)                                   0.454682
Jurassic Park (1993)                                                0.448910
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994)    0.445207
dtype: float64

In [8]:
#Rastgele bir film için işlemin yapılması
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)    1.000000
Ocean's Eleven (2001)                                      0.524170
American Beauty (1999)                                     0.493370
Fargo (1996)                                               0.490577
Monty Python and the Holy Grail (1975)                     0.471258
Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001)      0.467587
Kill Bill: Vol. 1 (2003)                                   0.446411
Ghostbusters (a.k.a. Ghost Busters) (1984)                 0.427393
Princess Bride, The (1987)                                 0.423259
One Flew Over the Cuckoo's Nest (1975)                     0.421177
dtype: float64

In [9]:
#Eğer filmin ismi tam olarak bilinmiyorsa
def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

check_film("Star Wars", user_movie_df)

['Star Wars: Episode I - The Phantom Menace (1999)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)']

In [11]:
##Kullanıcı Tabanlı İş Birlikçi Filtreleme

In [10]:
#Kütüphanelerin yüklenmesi
import pandas as pd
pd.set_option('display.max_columns', None) #Veri çerçevesi görüntülendiğinde tüm sütunların gösterilmesini sağlar.
pd.set_option('display.width', 15) #Veri çerçevesi görüntülenirken, konsolda kullanılan toplam genişliği ayarlar. Bu örnekte, genişlik 15 karakter olarak belirlenmiş.
pd.set_option('display.expand_frame_repr', False) #Veri çerçevesinin çok geniş olduğunda satırların bölünerek gösterilmesini engeller.
import numpy as np

In [11]:
# Excel dosyasının okunması
df = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Film.xlsx')

In [12]:
#İlk 5 satıra bak
df.head

<bound method NDFrame.head of            ID                              Title                        Original Title                                           Overview Release Date  Popularity  Vote Average  Vote Count                            Genres  UserId
0      533535               Deadpool & Wolverine                  Deadpool & Wolverine  Bütün sevdiklerini ve dünyasını tehdit eden bü...   2024-07-24    9866.173         7.785        2057      Aksiyon, Komedi, Bilim-Kurgu       1
1      718821                         Kasırgalar                              Twisters  Fırtına mevsimi yoğunlaştıkça, daha önce hiç g...   2024-07-10    4687.952         7.098         848    Aksiyon, Macera, Dram, Gerilim       1
2     1022789                         Ters Yüz 2                          Inside Out 2  Genç Riley'nin zihin merkezi, beklenmedik bir ...   2024-06-11    3210.402         7.602        2435   Animasyon, Aile, Macera, Komedi       1
3      519182                    Çılgın Hırsız

In [13]:
# Her film için toplam yorum sayısını (oy sayısını) hesapla
comment_counts = df[['Title', 'Vote Count']].groupby('Title').sum().reset_index()
# 100'den fazla oy sayısına sahip filmleri belirle
common_movies = comment_counts[comment_counts['Vote Count'] > 100]

# Çok yorum alan filmleri yorum sayısına göre azalan sırada sıralama
common_movies_sorted = common_movies.sort_values(by='Vote Count', ascending=False)

# Çok yorum alan filmleri ve yorum sayılarını yazdır
print("Çok Yorum Alan Filmler ve Yorum Sayıları (Yorum Sayısına Göre Sıralı):")
print(common_movies_sorted.head())

Çok Yorum Alan Filmler ve Yorum Sayıları (Yorum Sayısına Göre Sıralı):
                                Title  Vote Count
91                             Avatar       62321
406                    Geleceğe Dönüş       39194
147                         Başlangıç       36142
1232                   Yıldızlararası       34950
108   Açlık Oyunları: Ateşi Yakalamak       34178


In [14]:
# 1000'den az oy sayısına sahip filmleri belirle
rare_movies = comment_counts[comment_counts['Vote Count'] <= 100]

# Az yorum alan filmleri yorum sayısına göre sıralama
rare_movies_sorted = rare_movies.sort_values(by='Vote Count')

# Az yorum alan filmleri ve yorum sayılarını yazdır
print("Rare Movies ve Yorum Sayıları (Yorum Sayısına Göre Sıralı):")
print(rare_movies_sorted)

Rare Movies ve Yorum Sayıları (Yorum Sayısına Göre Sıralı):
                                  Title  Vote Count
1146                Una película barata           0
530                Joker: İkili Delilik           0
1367                ドクターZERO 精神分析医・財前零子           0
764               Mission: Impossible 8           0
766                             Moana 2           0
...                                 ...         ...
388                               Float          96
950                       Shadow Master          96
35                   Air Force One Down          97
94                        Avcının Kalbi          97
770   Mortal Kombat Legends: Cage Match          99

[384 rows x 2 columns]


In [15]:
# Kullanıcı ID'lerini alma
user_ids = df['UserId'].unique()

# Her kullanıcı için filmleri listeleme
for user_id in user_ids:
    user_movies = df[df['UserId'] == user_id]
    movies_watched = user_movies['Title'].tolist()
    print(f"Seçilen Kullanıcı ID: {user_id}")
    print(f"Kullanıcının İzlediği Filmler: {movies_watched}")

Seçilen Kullanıcı ID: 1
Kullanıcının İzlediği Filmler: ['Deadpool & Wolverine', 'Kasırgalar', 'Ters Yüz 2', 'Çılgın Hırsız 4', 'Bad Boys: Ya Hep Ya Hiç', 'Geber!', 'Sessiz Bir Yer: Birinci Gün', 'Alien: Romulus', 'Farang', 'Bizimle Başladı Bizimle Bitti', "Bikini Kasabası'nı Kurtarmak: Sandy Cheeks Filmi", 'Dragonkeeper', 'Prey', 'The Instigators', 'Garfield']
Seçilen Kullanıcı ID: 2
Kullanıcının İzlediği Filmler: ['Hız Peşinde', 'Furiosa: Bir Mad Max Destanı', 'The Convert', 'Maymunlar Cehennemi: Yeni Krallık', 'Paradox Effect', '還錢', '从21世纪安全撤离', "Ga'a Hanedanı", 'Thelma', "Mickey Mouse'ın Tuzağı", "Yeni Nesil: Kızıl'ın Yükselişi", 'Ev Ajanı', "Beni Ay'a Uçur", 'Gayribeyefendi Savaş Dairesi', 'Tuzak', 'Godzilla ve Kong: Yeni İmparatorluk', 'Justice League: Crisis on Infinite Earths Part Three', 'Çifte Pazarlık', 'Kung Fu Panda 4', 'Benim Hatam', 'Koralin ve Gizli Dünya', 'Un père idéal', 'Casusum: Ebedi Şehir', 'Deadpool', 'Yaratık', 'MaXXXine', 'İç Savaş']
Seçilen Kullanıcı ID: 3
Ku

In [16]:
# Rastgele bir kullanıcı ID'si seçme
random_user = int(pd.Series(user_ids).sample(1).values)

# Rastgele seçilen kullanıcının izlediği filmleri listeleme
print(f"Rastgele Seçilen Kullanıcı ID: {random_user}")
random_user_movies = df[df['UserId'] == random_user]['Title'].tolist()
print(f"Kullanıcının İzlediği Filmler: {random_user_movies}")

Rastgele Seçilen Kullanıcı ID: 5
Kullanıcının İzlediği Filmler: ['V de Víctor', 'Maymun Adam', 'Atlas', '劇場版 SPY×FAMILY CODE: White', 'Casusluk Yapacak Zaman Yok: Bir Loud House Filmi', 'Gözcüler', 'Borderlands', 'Jurassic World: Hâkimiyet', 'Hızlı ve Öfkeli 10', 'Yaratığın Dönüşü', 'Yenilmezler', 'Moana 2', "Bay Kleks'in Akademisi", 'Tetikte', 'Baki Hanma VS Kengan Ashura', 'Black Noise', 'Janet Planet', 'Motorcular', 'Avengers: Endgame', 'The Last Kumite', 'La Riffa', 'MR-9: Do or Die', 'Terrifier 3']


C:\Users\ilayd\AppData\Local\Temp\ipykernel_19728\2665131874.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  random_user = int(pd.Series(user_ids).sample(1).values)


In [17]:
#Rastgele bir kullanıcının seçilmesi
random_user = int(pd.Series(user_movie_df.index).sample(1).values)

#Kullanıcının yakalanması
random_user_df = user_movie_df[user_movie_df.index == random_user]

#Kullanıcının izlediği filmlerin (NaN olmayan) yakalanması
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

movies_watched

C:\Users\ilayd\AppData\Local\Temp\ipykernel_19728\3455519127.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  random_user = int(pd.Series(user_movie_df.index).sample(1).values)


['Die Hard (1988)',
 'Forrest Gump (1994)',
 'Fugitive, The (1993)',
 'Ghost (1990)',
 'Gladiator (2000)',
 'Good Will Hunting (1997)',
 'Goodfellas (1990)',
 'Green Mile, The (1999)',
 'Jurassic Park (1993)',
 'Matrix, The (1999)',
 "Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Sixth Sense, The (1999)',
 'Sleepless in Seattle (1993)',
 'Star Wars: Episode IV - A New Hope (1977)']

In [18]:
#DataFrame'in kullanıcının izlediği filmler ile filtrelenmesi
movies_watched_df = user_movie_df[movies_watched]

#Kullanıcıların bu filmlerden kaçını izlediği
user_movie_count = movies_watched_df.T.notnull().sum()

#Index resetleme
user_movie_count = user_movie_count.reset_index()

#Sütunları yeniden isimlendirmek
user_movie_count.columns = ["userId", "movie_count"]

In [19]:
#Kullanıcının izlediği bütün filmleri izleyen kullanıcı sayısı
user_movie_count[user_movie_count["movie_count"] == len(movies_watched)].count()

userId         8
movie_count    8
dtype: int64

In [22]:
#İzlediği film sayısının %60'ını bulmak
perc = len(movies_watched) * 60 / 100

#En az %60 oranında aynı filmi izleyen kullanıcıların id'sini yakalamak
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

In [23]:
#Kullanıcımız ve diğer kullanıcıların verilerini bir araya getirme
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)], random_user_df[movies_watched]])

In [27]:
# Sadece sayısal sütunları seçin
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Korelasyon matrisini oluşturun
corr_matrix = numeric_df.corr()

# Korelasyon matrisini düzene sokup sıralama
corr_df = corr_matrix.unstack().sort_values(ascending=False).drop_duplicates()

# Yüksek korelasyon gösteren ilk 10 sonucu listeleyin
high_corr = corr_df[corr_df < 1].head(10)

print(high_corr)

##Bu, oy sayısı ile oy ortalaması arasında orta dereceli bir pozitif ilişki olduğunu gösteriyor.Vote Count - Vote Average (0.357697)
##Popularity - ID (0.128070): Popülerlik ve ID arasında zayıf bir pozitif ilişki var. Bu, ID numarası büyük olan filmlerin genellikle daha popüler olduğunu, ancak bu ilişkinin çok güçlü olmadığını gösteriyor.
##Vote Count - UserId (-0.060547): Kullanıcı ID'si ile oy sayısı arasında çok zayıf bir negatif ilişki var. Bu, belirli kullanıcıların daha az oylama yaptığını gösterebilir.
##Vote Count - ID (-0.490578): Oy sayısı ile film ID'si arasında orta dereceli bir negatif ilişki var. Bu, daha yeni filmlerin genellikle daha az oy aldığını gösterebilir.

Vote Count    Vote Average    0.357697
Popularity    ID              0.128070
              Vote Average    0.039751
              Vote Count     -0.049140
UserId        Vote Average   -0.057734
Vote Count    UserId         -0.060547
UserId        ID             -0.211377
Vote Average  ID             -0.240279
Popularity    UserId         -0.362113
Vote Count    ID             -0.490578
dtype: float64


In [30]:
#korelasyon matrisi
print(corr_matrix.shape)
print(corr_matrix.head())

(5, 5)
                    ID  Popularity  Vote Average  Vote Count    UserId
ID            1.000000    0.128070     -0.240279   -0.490578 -0.211377
Popularity    0.128070    1.000000      0.039751   -0.049140 -0.362113
Vote Average -0.240279    0.039751      1.000000    0.357697 -0.057734
Vote Count   -0.490578   -0.049140      0.357697    1.000000 -0.060547
UserId       -0.211377   -0.362113     -0.057734   -0.060547  1.000000


In [39]:
# Sütun adlarının benzersiz olup olmadığını kontrol et
print(final_df.columns[final_df.columns.duplicated()])

# Satır adlarının benzersiz olup olmadığını kontrol et
print(final_df.index[final_df.index.duplicated()])

Index([], dtype='object', name='title')
Index([461], dtype='int64', name='userId')


In [40]:
print("Indexler:", final_df.index)
print("Sütunlar:", final_df.columns)

Indexler: Index([  1,
        17,
        18,
        28,
        42,
        45,
        57,
        62,
        63,
        64,
        66,
        68,
        84,
        91,
       103,
       105,
       135,
       140,
       144,
       156,
       160,
       166,
       177,
       182,
       198,
       199,
       200,
       202,
       219,
       220,
       232,
       239,
       247,
       249,
       254,
       263,
       274,
       275,
       282,
       288,
       292,
       304,
       305,
       307,
       313,
       317,
       328,
       330,
       339,
       352,
       354,
       357,
       368,
       380,
       381,
       385,
       387,
       391,
       414,
       415,
       419,
       425,
       432,
       434,
       438,
       448,
       453,
       461,
       462,
       464,
       469,
       474,
       477,
       480,
       483,
       489,
       503,
       514,
       517,
       525,
       555,
       561,
      

In [41]:
# Tekrarlanan satır indexlerini temizleme
final_df = final_df.loc[~final_df.index.duplicated(keep='first')]

In [42]:
# Korelasyon dataframe'inin oluşturulması
corr_df = final_df.T.corr().stack().sort_values().drop_duplicates()

# Sütun isimlendirmesi
corr_df = pd.DataFrame(corr_df, columns=["corr"])

# Çift index var, index isimlendirilmesi
corr_df.index.names = ['user_id_1', 'user_id_2']

# Index resetlenmesi
corr_df = corr_df.reset_index()

# İlk birkaç satırı göster
print(corr_df.head())

   user_id_1  user_id_2      corr
0        610        352 -0.866081
1        385         62 -0.852803
2        275        247 -0.850279
3        103        434 -0.849706
4        434        415 -0.844145


In [91]:
# Kullanıcı ID'lerinin bulunduğu kolonun adı
user_col = 'userId'

# Örnek bir kullanıcı ID'si belirleyelim (örneğin rastgele seçelim)
random_user = np.random.choice(corr_df['user_id_1'].unique())  # Korelasyon DataFrame'indeki mevcut kullanıcı ID'lerinden rastgele seçim

# Belirli kullanıcı ile yüksek korelasyonlu diğer kullanıcıları bulma
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop=True)

# Korelasyon df'inin sıralanması
top_users = top_users.sort_values(by='corr', ascending=False)

# Sütun ismi değiştirmek
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

print(f"Rastgele seçilen kullanıcı ID'si: {random_user}")
print(top_users.head())

Rastgele seçilen kullanıcı ID'si: 17
   userId      corr
6     198  0.863868
5     489  0.858395
4     586  0.817424
3     200  0.801324
2     562  0.794719


In [92]:
# Kullanıcıların film puanlarını içeren DataFrame'i filtreleme
recommended_movies = sorted_df[sorted_df['userId'].isin(top_users['userId'])]

# Her filmin ortalama puanını hesaplama
movie_recommendations = recommended_movies.groupby('title').agg({'rating': 'mean'}).reset_index()

# Filmleri yüksek puanlara göre sıralama
movie_recommendations = movie_recommendations.sort_values(by='rating', ascending=False)

print("Önerilen Filmler:")
print(movie_recommendations.head())

Önerilen Filmler:
                                       title  rating
679                      Jason Bourne (2016)    50.0
390   Dracula (Bram Stoker's Dracula) (1992)    50.0
872               Monsters University (2013)    50.0
406                       Dying Young (1991)    50.0
1306                    The Boss Baby (2017)    50.0
